In [1]:
#import sksurv
from sksurv.datasets import load_gbsg2
from survONS import surv_ons

x, y = load_gbsg2()

In [2]:
def cancer_levels(val):
    if val == "I":
        return 1
    elif val == "II":
        return 2
    elif val == "III":
        return 3
    else:
        return 0

In [3]:
x["horTh"] = x["horTh"].cat.codes
x["menostat"] = x["menostat"].apply(lambda val: 0 if (val == "Pre") else 1)
x["tgrade"] = x["tgrade"].apply(lambda val: cancer_levels(val))
x.head()

,age,estrec,horTh,menostat,pnodes,progrec,tgrade,tsize
0,70.0,66.0,0,1,3.0,48.0,2,21.0
1,56.0,77.0,1,1,7.0,61.0,2,12.0
2,58.0,271.0,1,1,9.0,52.0,2,35.0
3,59.0,29.0,1,1,4.0,60.0,2,17.0
4,73.0,65.0,0,1,1.0,26.0,2,35.0


In [4]:
import pandas as pd

df = pd.DataFrame(y)
df.head(10)

,cens,time
0,True,1814.0
1,True,2018.0
2,True,712.0
3,True,1807.0
4,True,772.0
5,True,448.0
6,False,2172.0
7,False,2161.0
8,True,471.0
9,False,2014.0


In [5]:
import numpy as np

N = x.shape[0]  #Número de individuosid
n_it = 2
max_t = df["time"].max()
d = x.shape[1]

eta = np.zeros((3,1))
sigma = np.eye(3,3)

gamma = np.arange(np.log(1/n_it), np.log(50), 1.2)
gamma = np.exp(gamma)

K = len(gamma)
M = 5

beta_real = np.zeros((d, M))
grid_bound = np.zeros((M, 1))

beta_surv = np.zeros((n_it,d,M))

like_surv = np.zeros((n_it, M))

gamma_t = np.zeros((n_it, M))

In [6]:
# for i in range(M):
#     beta_real[: , i] = np.random.normal(loc=0, scale=1, size=d)
#     D = 1.1 * np.cross(beta_real[:, i])
#     epsilon = 1/(gamma*D) ** 2
    
#     X = np.random.multivariate_normal(eta, sigma, N)

X = x.to_numpy()
t0 = np.zeros(N)
u = y["time"]
delta = y["cens"]
D = 2
n = n_it
epsilon = 1/(gamma*D) ** 2
R = [[] for _ in range(n_it)]

for t in range(n_it):
    # R[t].append(0)
    for i in range(N):
        t1 = 0
        t2 = min(max_t * t/n_it, u[i]) # asumiendo tiempos enteros
        R[t].append(0 if u[i] <= max_t * t/n_it else 1)

In [7]:
# a = np.array([1,1,1,1,1,1,1,1])
# b = np.eye(8)

In [8]:
survons = surv_ons(t0, u, delta, X, D, gamma, n, epsilon, R)
#print(survons[5])

beta 1
[[201.00664366   0.           0.           0.        ]
 [166.82297019   0.           0.           0.        ]
 [242.00081276   0.           0.           0.        ]
 [241.89294104   0.           0.           0.        ]
 [239.67428215   0.           0.           0.        ]
 [142.85795562   0.           0.           0.        ]
 [240.7617457    0.           0.           0.        ]
 [221.3011043    0.           0.           0.        ]]
var1 + -[201.00664366 166.82297019 242.00081276 241.89294104 239.67428215
 142.85795562 240.7617457  221.3011043 ]
status infeasible
beta 2
[[nan  0.  0.  0.]
 [nan  0.  0.  0.]
 [nan  0.  0.  0.]
 [nan  0.  0.  0.]
 [nan  0.  0.  0.]
 [nan  0.  0.  0.]
 [nan  0.  0.  0.]
 [nan  0.  0.  0.]]
beta 1
[[         nan 667.53033114   0.           0.        ]
 [         nan 554.03653837   0.           0.        ]
 [         nan 803.63576569   0.           0.        ]
 [         nan 803.27761897   0.           0.        ]
 [         nan 795.91141203   0.